In [1]:
import torch

In [2]:
from envs.othello.vectenv import OthelloVectEnv
from envs.othello.torchscripts import get_legal_actions, push_actions

env = OthelloVectEnv(100, torch.device('cpu'), 8)

/Users/marshingjay/Repos/lazyzero/envs/othello/torchscripts.py:84: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if x+1 < states_size:
/Users/marshingjay/Repos/lazyzero/envs/othello/torchscripts.py:85: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  flips[f_index, y, x, y, x+1:min(x+i, states_size)] = 1
/Users/marshingjay/Repos/lazyzero/envs/othello/torchscripts.py:86: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in

In [3]:
env.reset()
legal_actions = env.get_legal_actions()
actions = env.fast_weighted_sample(legal_actions.view(-1, 64).float())
env.push_actions(actions)
env.states[0]

tensor([[[0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 1, 1, 1, 0, 0],
         [0, 0, 0, 0, 1, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 1, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0]]])

In [63]:
env.next_turn()
legal_actions = env.get_legal_actions()
actions = env.fast_weighted_sample(legal_actions.view(-1, 64).float())
env.push_actions(actions)
env.states[0,0] + (2 * env.states[0,1]), actions[0]

(tensor([[2, 2, 2, 2, 2, 1, 1, 0],
         [2, 1, 1, 1, 1, 1, 1, 1],
         [2, 1, 2, 2, 2, 2, 2, 2],
         [2, 1, 2, 1, 2, 1, 2, 2],
         [2, 2, 1, 1, 2, 1, 2, 2],
         [2, 2, 1, 1, 2, 1, 2, 2],
         [1, 1, 2, 2, 1, 1, 2, 2],
         [1, 2, 2, 2, 2, 2, 2, 2]]),
 tensor(0))

In [64]:
env.next_turn()
legal_actions = env.get_legal_actions()
actions = env.fast_weighted_sample(legal_actions.view(-1, 64).float())
env.push_actions(actions)
env.states[0,1] + (2 * env.states[0,0]), actions[0]

(tensor([[2, 2, 2, 2, 2, 2, 2, 2],
         [2, 1, 1, 1, 1, 1, 2, 2],
         [2, 1, 2, 2, 2, 2, 2, 2],
         [2, 1, 2, 1, 2, 1, 2, 2],
         [2, 2, 1, 1, 2, 1, 2, 2],
         [2, 2, 1, 1, 2, 1, 2, 2],
         [1, 1, 2, 2, 1, 1, 2, 2],
         [1, 2, 2, 2, 2, 2, 2, 2]]),
 tensor(7))